## Face Tracking Frame by Frame

In [1]:
import cv2
import numpy as np
import torch
from torchvision import transforms, models
from PIL import Image
import pandas as pd


In [2]:
# Emotion mapping
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Loaded model
model = models.resnet34(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(emotion_dict)) 
model.load_state_dict(torch.load("resnet34_fer2013_weights30-6300.pth", map_location=device))
model = model.to(device)
model.eval()


Using device: cuda


d:\Program Files\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program Files\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Jimmy\AppData\Local\Temp\ipykernel_29076\596161222.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipp

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Set image transformations
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [5]:
# Activate webcam or load a video
cap = cv2.VideoCapture('test_video/0001.mp4')  # 0 -> Activate the webcam

# Enable face detection
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize result storage
frame_results = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Image preprocessing
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    frame_emotions = []  # Save the detection results for the current frame

    for (x, y, w, h) in faces:
        # Extract the face region
        face = gray_frame[y:y+h, x:x+w]
        face = Image.fromarray(face).convert('L')  # Convert to PIL format
        face_tensor = transform(face).unsqueeze(0).to(device)  # Add batch dimension and move to GPU/CPU

        # Predict emotions
        with torch.no_grad():
            outputs = model(face_tensor)
            _, predicted = torch.max(outputs, 1)
            emotion = emotion_dict[predicted.item()]

        # Record emotions for the current frame
        frame_emotions.append(emotion)

        # Display the results
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Save the emotions of the current frame
    frame_results.append(frame_emotions)

    cv2.imshow("Emotion Detection", frame)

    # Press 'q' to exit webcam (if used)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [6]:
cap.release()
cv2.destroyAllWindows()

In [8]:
# Convert results to DataFrame and save as CSV
df_results = pd.DataFrame(frame_results)
df_results.to_csv("emotion_analysis_results.csv", index=False)
print("Frame-by-frame emotion analysis results have been saved to emotion_analysis_results.csv")

Frame-by-frame emotion analysis results have been saved to emotion_analysis_results.csv
